In [1]:
using Pkg 
Pkg.instantiate()
using qAlgebra

using BenchmarkTools

Precompiling project...
    989.3 ms  ✓ qAlgebra
  1 dependency successfully precompiled in 1 seconds. 19 already precompiled.


In [2]:
qspace = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", operators=["A(!i)", "B(U,H,i)"], h=QubitPM(), i=(3, QubitPauli()), b=Ladder())

StateSpace: [β(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ, α]
   - SubSpace ["h"]: PM Qubit (Fermionic):  pₚ, mₚ, zₚ, Iₚ (identity)
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  xₚ, yₚ, zₚ, Iₚ (identity)
   - SubSpace ["b"]: Ladder (Bosonic):  p†, p
   - Op: A
   - Op: B(H,U)


In [3]:
var_dict, op_dict, abstract_dict = base_operators(qspace)
alpha = base_operators(qspace, "alpha")
beta = base_operators(qspace, "beta")
gamma_i, gamma_j, gamma_k = base_operators(qspace, "gamma", do_dict=false)
ph, mh, zh = base_operators(qspace, "h", do_dict=false)
xi,yi,zi, pi, mi = base_operators(qspace, "i", do_dict=false)
xj, yj, zj, pj, mj = base_operators(qspace, "j", do_dict=false)
xk, yk, zk, pk, mk = base_operators(qspace, "k", do_dict=false)
b, n = base_operators(qspace, "b", do_dict=false)
I = base_operators(qspace, "I")
A = base_operators(qspace, "A")
println("Done")

Done


In [4]:
A = base_operators(qspace, "A", do_fun=true)
A1 = base_operators(qspace, "A_1", do_fun=false)
A = A()

A

In [5]:
Sum("j", alpha * yi * yj + Sum("k", beta * alpha^2 * xi * xi * xk))

∑ⱼ⁼(αyᵢyⱼ+∑ₖ⁼β(t)α²xₖ)

In [6]:
@benchmark Sum("j", alpha * yi * yj + Sum("k", beta * alpha^2 * xi * xj * xk))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  69.292 μs …   9.639 ms  ┊ GC (min … max): 0.00% … 98.71%
 Time  (median):     70.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   74.791 μs ± 144.214 μs  ┊ GC (mean ± σ):  3.25% ±  1.71%

     ▃██▆                                                       
  ▁▂▄█████▆▃▂▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  69.3 μs         Histogram: frequency by time         83.3 μs <

 Memory estimate: 48.80 KiB, allocs estimate: 807.

In [7]:
simplify(flatten(Sum("j", alpha * yi * yj + Sum("k", beta * alpha^2 * xi * xj * xk))))

∑ⱼ⁼αyᵢyⱼ+∑ⱼₖ⁼β(t)α²xᵢxⱼxₖ

In [10]:
expr = neq(log(Sum("j", alpha * yi * yj + Sum("k", beta * gamma_j * gamma_k * xi * xj * xk))))

log(α+2β(t)γᵢ²xᵢ+∑ⱼ(αyᵢyⱼ+β(t)γⱼ²xᵢ+β(t)γᵢγⱼxⱼ)+∑ₖβ(t)γᵢγₖxₖ+∑₍ⱼₖ₎β(t)γⱼγₖxᵢxⱼxₖ)

In [11]:
d_dt(xi, expr)

d(xᵢ) / dt = log(α+2β(t)γᵢ²xᵢ+∑ⱼ(αyᵢyⱼ+β(t)γⱼ²xᵢ+β(t)γᵢγⱼxⱼ)+∑ₖβ(t)γᵢγₖxₖ+∑₍ⱼₖ₎β(t)γⱼγₖxᵢxⱼxₖ)

In [12]:
function extract_qabstract(q::qExpr)::qAbstract
    if length(q) > 1 
        error("Cannot substitute composites. abstract_op must contain only an abstract operator.")
    end
    term = q.terms[1]
    if !isa(term, qAtomProduct)
        error("qExpr must contain only a qAtomProduct")
    end
    return extract_qabstract(term)
end
function extract_qabstract(term::qAtomProduct)::qAbstract
    if length(term.expr) != 1 || !isa(term.expr[1], qAbstract)
        error("abstract_op must contain exactly one qAbstract")
    end
    return term.expr[1]
end
#now same for qAtom 
function extract_qatom(q::qExpr)::qAtom
    if length(q) > 1 
        error("Cannot substitute composites. abstract_op must contain only an abstract operator.")
    end
    term = q.terms[1]
    if !isa(term, qAtomProduct)
        error("qExpr must contain only a qAtomProduct")
    end
    return extract_qatom(term)
end
function extract_qatom(term::qAtomProduct)::qAtom
    if length(term.expr) != 1 || !isa(term.expr[1], qAtom)
        error("abstract_op must contain exactly one qAbstract")
    end
    return term.expr[1]
end

extract_qatom (generic function with 2 methods)

In [13]:
# substitute abstract operator 
# input is abstract_op, replacement and target
simpleQ = Union{qExpr, qAtomProduct}
function substitute(abstract_op::Union{simpleQ, qAbstract}, replacement::Union{simpleQ, qAtom}, target::qExpr)::qExpr
    if !isa(abstract_op, qAbstract)
        abstract_op = extract_qabstract(abstract_op)
    end
    if !isa(replacement, qAtom)
        replacement = extract_qatom(replacement)
    end
    return substitute(abstract_op, replacement, target)
end
function substitute(abstract_op::qAbstract, replacement::qAtom, target::qExpr)::qExpr
    # recursively navigate expression, and substitue
    new_terms = qComposite[]
    for term in target.terms
        append!(new_terms, substitute(abstract_op, replacement, term))
    end
    return qExpr(target.statespace, new_terms)
end
function substitute(a::qAbstract, r::qAtom, targ::T) where T<:qComposite
    # T<:qMultiComposite is *also* <:qComposite, 
    # so we need the qMultiComposite method to be more specific
    cp = copy(targ)
    cp.expr = substitute(a, r, targ.expr)
    return [cp]  # Tuple or Vector, depending on your convention
end

# For anything that holds *many* sub‑expressions
function substitute(a::qAbstract, r::qAtom, targ::T) where T<:qMultiComposite
    cp = copy(targ)
    cp.expr = map(x -> substitute(a, r, x), targ.expr)
    return [cp]
end
function substitute(abstract_op::qAbstract, replacement::qAtom, target::qAtomProduct)::Vector{qComposite}
    # recursively navigate expression, and substitue
    expr = target.expr
    new_expr::Vector{qAtom} = []
    for t in expr
        if isa(t, qTerm)
            push!(new_expr, t)
        else# qAbstract
            # check if its the same qAbstract operator 
            if t.key_index == abstract_op.key_index && t.sub_index == abstract_op.sub_index 
                if t.exponent != 1
                    curr_replacement = replacement^t.exponent
                    if t.dag 
                        curr_replacement = curr_replacement'
                    end
                    push!(new_expr, curr_replacement)
                elseif t.dag
                    push!(new_expr, replacement')
                else
                    push!(new_expr, replacement)
                end
            else
                push!(new_expr, t)
            end
        end
    end
    return simplify(qAtomProduct(target.statespace, target.coeff_fun, new_expr))
end
function substitute(abstract_op::Union{simpleQ, qAbstract}, replacement::Union{simpleQ, qAtom}, target::diff_qEQ)::diff_qEQ 
    if !isa(abstract_op, qAbstract)
        abstract_op = extract_qabstract(abstract_op)
    end
    if !isa(replacement, qAtom)
        replacement = extract_qatom(replacement)
    end
    return substitute(abstract_op, replacement, target)
end
function substitute(abstract_op::qAbstract, replacement::qAtom, target::diff_qEQ)::diff_qEQ
    lhs = substitute(abstract_op, replacement, target.left_hand_side)
    if length(lhs) != 1
        error("Substitution of $abstract_op with $replacement in $target did not result in a single term.")
    end
    lhs = lhs[1]
    rhs = substitute(abstract_op, replacement, target.expr)
    return diff_qEQ(target.statespace, 
        lhs,
        rhs, 
        target.braket, 
        target.do_sigma)
end

substitute (generic function with 7 methods)

In [14]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))

log(αyᵢA+2β(t)γᵢ²xᵢAxᵢ+∑ⱼ(αyⱼA+β(t)γⱼ²xⱼAxⱼ+β(t)γᵢγⱼxⱼAxᵢ)+∑ₖβ(t)γᵢγₖxᵢAxₖ+∑₍ⱼₖ₎β(t)γⱼγₖxⱼAxₖ)

In [15]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))
dA_dt = d_dt(A, expr)
substitute(A, xi, dA_dt)

d(xᵢ) / dt = log(αyᵢxᵢ+2β(t)γᵢ²xᵢxᵢxᵢ+∑ⱼ(αyⱼxᵢ+β(t)γⱼ²xⱼxᵢxⱼ+β(t)γᵢγⱼxⱼxᵢxᵢ)+∑ₖβ(t)γᵢγₖxᵢxᵢxₖ+∑₍ⱼₖ₎β(t)γⱼγₖxⱼxᵢxₖ)

In [16]:
using BenchmarkTools

@benchmark substitute(A, xi, dA_dt)

BenchmarkTools.Trial: 10000 samples with 6 evaluations per sample.
 Range (min … max):  5.667 μs …  1.840 ms  ┊ GC (min … max): 0.00% … 99.44%
 Time  (median):     5.993 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.428 μs ± 22.114 μs  ┊ GC (mean ± σ):  4.76% ±  1.40%

       ▁▄▆▆█▅▂▁                                               
  ▁▂▃▄▆████████▆▆▅▄▄▅▄▆▆▆▅▅▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  5.67 μs        Histogram: frequency by time        7.26 μs <

 Memory estimate: 8.30 KiB, allocs estimate: 142.

In [17]:
# check if all abstracts are gone
function contains_abstract(term::qExpr)::Bool
    return any([contains_abstract(t) for t in term.terms])
end
function contains_abstract(term::T)::Bool where T<:qComposite
    return contains_abstract(term.expr)
end
function contains_abstract(term::T)::Bool where T<:qMultiComposite
    return any([contains_abstract(t) for t in term.expr])
end
function contains_abstract(term::qAtomProduct)::Bool
    return any([isa(t, qAbstract) for t in term.expr])
end
# Test 
println("Before: ", contains_abstract(expr)) 
println("After:  ", contains_abstract(substitute(A, xi, dA_dt)))

Before: true
After:  false


In [18]:
qspace.where_by_continuum

Dict{Int64, Vector{Vector{Int64}}} with 1 entry:
  2 => [[2, 3, 4], [5, 6, 7]]

1-element Vector{Int64}:
 2

In [ ]:
Is = Union{Int,Vector{Int}}
function vecvec_or(A::Vector{Vector{Bool}}, B::Vector{Vector{Bool}})
    # Assume they are equally shaped. 
    return broadcast.(|, A, B)
end
cnimp(a::Bool, b::Bool) = b && !a
function converse_nonimplication(A::Vector{Vector{Bool}}, B::Vector{Vector{Bool}})
    # Assume they are equally shaped. 
    return broadcast.(cnimp, A, B) 
end

# FFunctions
function which_continuum_acting(f::FAtom, where_continuums_f::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    where_non_trivial::Vector{Vector{Bool}} = []
    for where_f in where_continuums_f
        push!(where_non_trivial, reduce(.|, [f.var_exponents[w] .!= 0 for w in where_f]))
    end
    return where_non_trivial
end
function which_continuum_acting(f::FSum, where_continuums_f::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    # or of the individual terms 
    return reduce(vecvec_or, [which_continuum_acting(t, where_continuums_f) for t in f.terms])
end
function which_continuum_acting(f::FRational, where_continuums_f::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    return vecvec_or(which_continuum_acting(f.num, where_continuums_f), which_continuum_acting(f.den, where_continuums_f))
end

# QObjs
function which_continuum_acting(q::qAtom, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Vector{Vector{Bool}}
    my_continuums::Vector{Vector{Bool}} = []
    for (inds, neutral) in zip(continuum_indexes, neutral_continuums_op)
        push!(my_continuums, q.op_indices[inds] .!= neutral)
end
function which_continuum_acting(q::qAbstract, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})
    error("Which continuum acting should be applied to abstractless expressions!")
end
function which_continuum_acting(q::qAtomProduct, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}}, variable_indexes::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    # xor between vectors of vector of bool 
    return vecvec_or(reduce(vecvec_or, [which_continuum_acting(t, continuum_indexes, neutral_continuums_op) for t in q.terms]), which_continuum_acting(q.coeff_fun, variable_indexes))
end

In [ ]:
function are_indexes_defined(q::qAtomProduct, where_defined::Vector{Vector{Bool}}, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Bool
    # check that no true on which_continuum_acting, that isn't also a true on where_defined => converse nonimplication cnimp(a::Bool, b::Bool) = b && !a
    if converse_nonimplication(where_defined, which_continuum_acting(q, continuum_indexes, neutral_continuums_op))
        error("Cannot use an undefined continuumsindex on the right hand side of a differential equation!")
    end
    return true
end
function are_indexes_defined(q::qExpr, where_defined::Vector{Vector{Bool}}, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Bool
    # check element wise if all indexes are defined
    return all([are_indexes_defined(t, where_defined, continuum_indexes, neutral_continuums_op) for t in q.terms])
end
function are_indexes_defined(q::T, where_defined::Vector{Vector{Bool}}, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Bool where T <:qComposite
    return are_indexes_defined(q.expr, where_defined, continuum_indexes, neutral_continuums_op)
end
function are_indexes_defined(q::T, where_defined::Vector{Vector{Bool}}, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Bool where T <:qMultiComposite
    return all([are_indexes_defined(t, where_defined, continuum_indexes, neutral_continuums_op) for t in q.exprs])
end
function are_indexes_defined(q::qSum, where_defined::Vector{Vector{Bool}}, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Bool
    # add the qSum summation indexes 
    subsystem = q.subsystem_index 
    element_indexes = q.element_indexes
    # check where subsystem is among qspace.where_continuum
    outer_ind = findfirst(x -> x == subsystem, continuum_indexes)
    if !all(where_defined[outer_ind][element_indexes] .== false)
        error("Summation indexes already defined, cannot sum over defined indexes!")
    end
    new_where_defined = copy(where_defined)
    new_where_defined[outer_ind][element_indexes] .= true
    # check the summation qExpr 
    return are_indexes_defined(q.expr, new_where_defined, continuum_indexes, neutral_continuums_op)
end
function are_indexes_defined(q::diff_qEQ)::Bool
    qspace = q.statespace
    # first two arguments for operators 
    # final argument for paramete/variables
    defined = which_continuum_acting(q.left_hand_side, qspace.continuum_indexes, qspace.neutral_continuum_op, qspace.where_by_continuum_vec)
    # check recursively if there are undefined elements in the right hand side
    return are_indexes_defined(q.expr, defined, qspace.continuum_indexes, qspace.neutral_continuum_op)
end

In [ ]:
# in diff equations can't sum over indexes of the left side.  -> throw an error
# reduce summed indexes to lowest summable indexes in the right side.
# there can also not be any variables of non left indexes outside of such sums. 

# add evaluate 
# add index version of qAtomProduct.

# Add Equation Set
# Add Equation Set to Indexed Equation Set with indexed  

In [7]:
flatten(Sum("j", alpha * yi * yj + qCommutator(Sum("k", beta * alpha^2 * xi * xj * xk), xk)))

ErrorException: Unsupported: qSum found inside qComposite structure within an outer qSum.

In [7]:
Sum(["i", "i"], zi) 

∑ᵢᵢ⁼zᵢ

In [8]:
true && false

false

In [9]:
Sum("i", alpha*ph*xi*yi * A * xi + xi*yi) * Sum("j", zi) 

MethodError: MethodError: no method matching *(::qExpr, ::qAlgebra.qExpressions.var"#28#30"{StateSpace, OperatorType, Int64})
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  *(::qExpr, !Matched::qExpr)
   @ qAlgebra ~/Documents/PhD/Research/Projects/qAlgebra/src/qExpressionsOps/qExpressionsAlgebra.jl:222
  *(::qExpr, !Matched::Vector{qExpr})
   @ qAlgebra ~/Documents/PhD/Research/Projects/qAlgebra/src/qExpressionsOps/qExpressionsAlgebra.jl:335
  ...


In [11]:
2 * (alpha + beta) * xi*1im + alpha * Dag(b) * xi * yi

iαzᵢb†+2i(α+β(t))xᵢ

In [12]:
alpha*(beta+beta^2)*1im

iβ(t)α+iβ(t)²α

In [13]:
d_dt(A, xi*A'*A + alpha*b*b')

MethodError: MethodError: no method matching adjoint(::qAlgebra.qExpressions.var"#28#30"{StateSpace, OperatorType, Int64})
The function `adjoint` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  adjoint(!Matched::FAtom)
   @ qAlgebra ~/Documents/PhD/Research/Projects/qAlgebra/src/FFunctions.jl:372
  adjoint(!Matched::Missing)
   @ Base missing.jl:101
  adjoint(!Matched::ComplexRationals.ComplexRational)
   @ ComplexRationals ~/.julia/packages/ComplexRationals/liwaH/src/ComplexRationals.jl:266
  ...


In [9]:
simplify(qCommutator(Sum("i", alpha*ph*xi*yi) + zj,zh))

[zⱼ+∑ᵢ⁼iαpₕzᵢ, zₕ]

In [10]:
simplify(exp(Sum("i", alpha*ph*xi*yi) + zj)+zh)

zₕ+exp(zⱼ+∑ᵢ⁼iαpₕzᵢ)

In [11]:
log(Sum("i", alpha*ph*xi*yi) + zj)

log(zⱼ+∑ᵢ⁼iαpₕzᵢ)

In [12]:
simplify(power(Sum("i", alpha*ph*xi*yi) + zj,2)+zh)

zₕ+(zⱼ+∑ᵢ⁼iαpₕzᵢ)²

In [13]:
simplify(root(Sum("i", alpha*ph*xi*yi) + zj,2)+zh)

zₕ+(zⱼ+∑ᵢ⁼iαpₕzᵢ)⁼²

In [14]:
Sum("i", alpha*ph*xi*yi * A * xi + xi*yi) * Sum("j", zi) 

∑ᵢ⁼(izᵢ+iαpₕzᵢAxᵢ)∑ⱼ⁼zᵢ

In [ ]:
# Build and view documentation
julia --project=. -e 'include("docs/make.jl")'
xdg-open docs/build/index.html


In [3]:
A = base_operators(qspace, "A_2")
alpha = base_operators(qspace, "alpha")
A*alpha*A

α*A₂²

In [4]:
alpha^2*A^2*alpha*A

α³*A₂³